In [1]:
import sys
sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')
import importlib

from __init__ import *
from __init__jupyterlab import *
import snmcseq_utils
importlib.reload(snmcseq_utils)
import CEMBA_preproc_utils
importlib.reload(CEMBA_preproc_utils)
import CEMBA_clst_utils

from scipy.io import mmread
from scipy import sparse
import time

import fbpca

In [5]:
log = snmcseq_utils.create_logger()

In [7]:
logging.info('0')
f = '/cndd/Public_Datasets/AIBS_mouse_VISp/mouse_VISp_2018-06-14_exon-matrix.csv'
df_exon = pd.read_csv(f, index_col=0)
logging.info('1')

f = '/cndd/Public_Datasets/AIBS_mouse_VISp/mouse_VISp_2018-06-14_intron-matrix.csv'
df_intron = pd.read_csv(f, index_col=0)
logging.info('2')
print(df_exon.shape, df_intron.shape)


08/14/2019 02:40:19 PM 0
08/14/2019 02:43:15 PM 1
08/14/2019 02:46:14 PM 2


(45768, 15413) (45768, 15413)


In [8]:
f = '/cndd/Public_Datasets/AIBS_mouse_VISp/mouse_VISp_2018-06-14_samples-columns.csv'
meta = pd.read_csv(f, index_col=0)
print(meta.shape)

f = '/cndd/Public_Datasets/AIBS_mouse_VISp/mouse_VISp_2018-06-14_genes-rows.csv'
df_genes = pd.read_csv(f, index_col=0)
print(df_genes.shape)

f = '/cndd/Public_Datasets/AIBS_mouse_VISp/rsem_GRCm38.p3.gtf'
df_genes_2 = pd.read_csv(f, index_col=0)
print(df_genes_2.shape)

(15413, 49)
(45768, 4)
(1278219, 0)


In [18]:
counts = df_exon + df_intron
counts.index = (df_genes.reset_index()
                     .set_index('gene_entrez_id')
                     .loc[counts.index, 'gene_symbol'].values
                )
assert np.unique(counts.index.values).shape == counts.index.values.shape

print(counts.shape)
counts.head()

(45768, 15413)


,F1S4_160108_001_A01,F1S4_160108_001_B01,F1S4_160108_001_C01,F1S4_160108_001_D01,F1S4_160108_001_E01,F1S4_160108_001_F01,F1S4_160108_001_G01,F1S4_160108_001_H01,F1S4_160108_002_A01,F1S4_160108_002_B01,...,FYS4_171004_103_G01,FYS4_171004_103_H01,FYS4_171004_104_A01,FYS4_171004_104_B01,FYS4_171004_104_C01,FYS4_171004_104_D01,FYS4_171004_104_E01,FYS4_171004_104_F01,FYS4_171004_104_G01,FYS4_171004_104_H01
0610005C13Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610006L08Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610007P14Rik,79,123,89,115,402,266,82,166,118,121,...,264,45,88,81,187,118,102,134,112,37
0610009B22Rik,145,178,69,0,208,111,154,60,76,114,...,143,82,79,105,52,166,175,269,184,93
0610009E02Rik,1,26,0,0,0,0,5,0,7,88,...,0,0,0,0,0,0,0,0,25,0


In [49]:
cell_class = [
    'Glutamatergic', 
#     'GABAergic', 
    ]

meta = meta[meta['class'].isin(cell_class)]

In [50]:
# meta.groupby(['class', 'subclass', 'cluster']).size()
# meta[meta['subclass']=='CR']

In [51]:
counts = counts[meta.index.values]

In [52]:
print(meta.shape)
print(counts.shape)

(7366, 49)
(45768, 7366)


In [53]:
gxc_mat = GC_matrix(
    counts.index.values,
    counts.columns.values,
    sparse.coo_matrix(counts.values),
)

In [54]:
mod = 'smarter-cells-v1'
DATADIR = '/cndd/fangming/CEMBA/data/MOp_all/organized_mop_v1_exc'

f_meta = os.path.join(DATADIR, '{}_metadata.tsv'.format(mod))
meta.to_csv(f_meta, sep="\t", na_rep='NA', header=True, index=True)

f_mat = os.path.join(DATADIR, '{}_raw.npz'.format(mod))
f_gene = os.path.join(DATADIR, '{}_raw.gene'.format(mod))
f_cell = os.path.join(DATADIR, '{}_raw.cell'.format(mod))

snmcseq_utils.save_gc_matrix(gxc_mat, f_gene, f_cell, f_mat)

# SMARTER-cells MOP

In [66]:
f = '/cndd/fangming/CEMBA/data/MOp_all/organized_neurons_v8/smarter_cells_metadata.tsv'
meta = pd.read_csv(f, sep="\t", index_col=0)
meta = meta[meta['Class_Allen'].isin(cell_class)]
print(meta.shape)
meta.head()

(4152, 10)


,Category_Allen,Class_Allen,ClusterID_Allen,ClusterID_observed_n50_k30,ClusterName_Allen,Subclass_Allen,Valid_Neuron,Valid_Non-Neuronal,Valid_QC,Reads_Unique
CellID,,,,,,,,,,
LS-15395_S43_E1-50,Neuron,Glutamatergic,46,22,L6 NP Trh,L5/6 NP,1,0,True,1761973
LS-15395_S44_E1-50,Neuron,Glutamatergic,45,17,L5 NP Slc17a8,L5/6 NP,1,0,True,2058419
LS-15395_S46_E1-50,Neuron,Glutamatergic,38,5,L5 IT S100b_2,L5 IT,1,0,True,2093078
LS-15395_S48_E1-50,Neuron,Glutamatergic,38,5,L5 IT S100b_2,L5 IT,1,0,True,1918645
LS-15395_S50_E1-50,Neuron,Glutamatergic,38,5,L5 IT S100b_2,L5 IT,1,0,True,1869016


In [67]:
f_gene = '/cndd/fangming/CEMBA/data/MOp_all/organized_neurons_v8/smarter_cells_raw.gene'
f_cell = '/cndd/fangming/CEMBA/data/MOp_all/organized_neurons_v8/smarter_cells_raw.cell'
f_mat = '/cndd/fangming/CEMBA/data/MOp_all/organized_neurons_v8/smarter_cells_raw.npz'
gc_mat = snmcseq_utils.load_gc_matrix(f_gene, f_cell, f_mat)

In [68]:
new_cells = meta.index.values
cell_idx = snmcseq_utils.get_index_from_array(gc_mat.cell, new_cells)

In [69]:
gxc_mat = GC_matrix(
    gc_mat.gene, 
    new_cells,
    gc_mat.data.tocsc()[:, cell_idx],
)

In [70]:
mod = 'smarter_cells'
DATADIR = '/cndd/fangming/CEMBA/data/MOp_all/organized_mop_v1_exc'

f_meta = os.path.join(DATADIR, '{}_metadata.tsv'.format(mod))
meta.to_csv(f_meta, sep="\t", na_rep='NA', header=True, index=True)

f_mat = os.path.join(DATADIR, '{}_raw.npz'.format(mod))
f_gene = os.path.join(DATADIR, '{}_raw.gene'.format(mod))
f_cell = os.path.join(DATADIR, '{}_raw.cell'.format(mod))

snmcseq_utils.save_gc_matrix(gxc_mat, f_gene, f_cell, f_mat)